# Assignment 4.9 Part 1

### This Script Contains the Following:
##### 3. Importing Data and Libraries
##### 4. Exploring Data and Data wrangling
##### 5. Data Quality and Consistency Checks
##### 6. Combining Dataframes
##### 8. Exporting Dataframes

#### 3. Importing Data and Libraries

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# create string
path = r'C:\Users\Jamie&Ken\Desktop\Data Analytics Course\Python Fundeamentals for Data Analysts\08-2024 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.9_ords_prods_merge.pkl'))
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

#### 4. Exploring Data and Data Wrangling

In [4]:
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
cust.tail()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799
206208,80148,Cynthia,Noble,Female,New York,55,4/1/2020,1,married,57095


In [6]:
cust.shape

(206209, 10)

In [7]:
cust.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [8]:
cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


###### No missing data found in head or tail, will investigate further in consistency checks. Data types look okay, will change user_id to string as does not need to be included in df.describe. General details in the describe function look okay. Names of columns are all in different formats

In [9]:
# Change user id dtype
cust['user_id'] = cust['user_id'].astype('str')

In [10]:
#Check dtype
cust['user_id'].dtype

dtype('O')

In [11]:
cust.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [12]:
# rename columns to be of similar formating
cust.rename(columns = {'First Name' : 'first_name'}, inplace = True)
cust.rename(columns = {'Surnam' : 'last_name'}, inplace = True)
cust.rename(columns = {'Gender' : 'gender'}, inplace = True)
cust.rename(columns = {'STATE' : 'state'}, inplace = True)
cust.rename(columns = {'Age' : 'age'}, inplace = True)
cust.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)
cust.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

In [13]:
# Check columns renamed properly
cust.columns

Index(['user_id', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'number_of_dependants', 'marital_status', 'income'],
      dtype='object')

#### 5. Data Quality and Consistency Checks

In [14]:
# Checking for mixed data types
for col in cust.columns.tolist():
    weird = (cust[[col]].map(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (cust[weird]) > 0:
        print (col)

first_name


###### The first_name column has a mixed data type that needs to be changed

In [15]:
# Change data type of first_name
cust['first_name'] = cust['first_name'].astype('str')

In [16]:
# Check to make sure data type changed
for col in cust.columns.tolist():
    weird = (cust[[col]].map(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (cust[weird]) > 0:
        print (col)

###### no result = no mixed data type

In [17]:
# Checking for missing values
missing_values = cust.isnull().sum()

In [18]:
missing_values

user_id                 0
first_name              0
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependants    0
marital_status          0
income                  0
dtype: int64

In [19]:
# As first_name's data type was changed str above, a different approach must be used
missing_values = (cust['first_name'] == 'nan').sum()
missing_values

11259

In [20]:
cust['first_name'].value_counts(dropna = False)

first_name
nan        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: count, Length: 208, dtype: int64

In [21]:
# Create a mask indicating which rows contain 'nan'
mask = cust['first_name'] == 'nan'
missing_cust = cust[mask]

In [22]:
# check data
missing_cust.head(10)

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income
53,76659,nan,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,nan,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,nan,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,nan,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,nan,Dawson,Female,Utah,63,1/1/2017,3,married,151819
128,8562,nan,Oconnor,Male,Utah,46,1/1/2017,1,married,134898
140,149267,nan,Hutchinson,Male,South Carolina,20,1/1/2017,0,single,86778
149,82632,nan,Orr,Male,Hawaii,61,1/1/2017,1,married,118130
155,172331,nan,Williamson,Female,Alaska,27,1/1/2017,0,single,55047
236,182963,nan,Nicholson,Female,New Mexico,58,1/2/2017,1,married,163391


###### Missing data is over 5% so should not be removed, it is not numeric so we cannot imput values, therefore it should be left. If client gets back to us and states data can be dropped, we can remove.

In [23]:
# Check for dupliactes
cust[cust.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income


###### No dupliactes found

#### 6. Combining Dataframes 

###### Dataframe to be combined imported in first step

In [24]:
# finding common column
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,weekday_of_order,order_hour_of_day,...,busiest_period_of_day,max_order,loyalty_flag,average_spender,average_spende,average_spend,spender_flag,frequent_customer,customer_frequency,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Most orders,32,Regular customer,6.935811,6.935811,6.935811,Low spender,8.0,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Average orders,32,Regular customer,6.935811,6.935811,6.935811,Low spender,8.0,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Average orders,5,New customer,7.930208,7.930208,7.930208,Low spender,8.0,8.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Most orders,3,New customer,4.972414,4.972414,4.972414,Low spender,9.0,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Average orders,3,New customer,4.972414,4.972414,4.972414,Low spender,9.0,9.0,Frequent customer


In [25]:
# confirm data type for columns
print(ords_prods_merge['user_id'].dtype)
print(cust['user_id'].dtype)

int64
object


In [26]:
# convert user_id in cust dataframe to numeric
cust['user_id'] = pd.to_numeric(cust['user_id'], errors='coerce')

In [29]:
# Merge dataframes
df_merged = pd.merge(ords_prods_merge, cust, on = 'user_id', how = 'inner')

In [30]:
df_merged.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,weekday_of_order,order_hour_of_day,...,frequency_flag,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Frequent customer,Deborah,Glass,Female,Vermont,66,6/16/2018,2,married,158302
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Frequent customer,Heather,Myers,Female,Wisconsin,40,2/9/2020,3,married,31308
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Frequent customer,Heather,Myers,Female,Wisconsin,40,2/9/2020,3,married,31308


In [32]:
df_merged.shape

(32404859, 36)

In [33]:
df_merged.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'weekday_of_order',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'add_to_cart_order', 'reordered', '_merge', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'average_spender', 'average_spende', 'average_spend', 'spender_flag',
       'frequent_customer', 'customer_frequency', 'frequency_flag',
       'first_name', 'last_name', 'gender', 'state', 'age', 'date_joined',
       'number_of_dependants', 'marital_status', 'income'],
      dtype='object')

In [37]:
df_merged['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [38]:
df_merged['average_spende'].value_counts()

average_spende
8.000000     10344
7.500000      9116
8.100000      8232
7.400000      7044
7.750000      7040
             ...  
11.933333        3
13.800000        3
12.033333        3
9.733333         3
8.066667         3
Name: count, Length: 128623, dtype: int64

In [39]:
df_merged['average_spend'].value_counts()

average_spend
8.000000     10344
7.500000      9116
8.100000      8232
7.400000      7044
7.750000      7040
             ...  
11.933333        3
13.800000        3
12.033333        3
9.733333         3
8.066667         3
Name: count, Length: 128623, dtype: int64

In [45]:
# Remove excess column 
df_merged.drop(['average_spende', 'frequent_customer'], axis=1, inplace=True)

In [46]:
df_merged.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'weekday_of_order',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'add_to_cart_order', 'reordered', '_merge', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'average_spender', 'average_spend', 'spender_flag',
       'customer_frequency', 'frequency_flag', 'first_name', 'last_name',
       'gender', 'state', 'age', 'date_joined', 'number_of_dependants',
       'marital_status', 'income'],
      dtype='object')

In [48]:
df_merged.shape

(32404859, 34)

#### 8. Exporting Dataframes

In [49]:
#export combined df as pickle
df_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ord_prods_cust_merge.pkl'))